In [12]:
!pip install spotipy

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import time
import datetime
import os

client_id='e08812d0ad874b53bb01e49ada6166ad'
client_secret='9085378d43d24d6c823a92c0f2cb03b5'

# # 민진님
# client_id = "7a98ceda33624c9bb15bf7f877176e50"
# client_secret = "74fb60bce9d048e3a9faa52427e6d0f5"

# # 세인님
# client_id = "05c96c56746e49b9b26e2ccf9837c7b7"
# client_secret = "98238151cd5e46df93049946ddd3673a"

# # 서브 1
# client_id = "55cc1bbf202640ee8a691ecab9be6e1d"
# client_secret = "ef892d15d19543f4840c5ff069e6503c"

# # 서브 2
# client_id = "37211c9678ae40e6aed26be454135b1e"
# client_secret = "031a960bb8bc4367a412f63d25f1f513"

# # 서브 3
# client_id = "4cda4cd5cf9a4877b55336bedbe21c9c"
# client_secret = "93bacd33bd494ec5bc76034f3705d502"

# # 서브 4
# client_id = "9e6e9e6622204dbe85e2553c9b026ed5"
# client_secret = "b7b96ff10fae4077ab8912e1fb9d1f6b"

# # 서브 5
# client_id = "8e564786163d4a66a8e8d844955b9c2d"
# client_secret = "7966e74a259d4fd9a3d689b54fd3d100"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [14]:
# 검토 완료 진행상황 표시

start_time = time.time()

temporary_df = pd.DataFrame()  # 빈 DataFrame 생성
temporary_df2 = pd.DataFrame()  # 빈 DataFrame 생성
temporary_df3 = pd.DataFrame() 

artist_cache = {}  # 아티스트 정보 캐시 생성

year_list = ['2023']
genre_list = ['edm', 'disco', 'electronic']

total_workload = len(year_list) * len(genre_list) * 100  # 100은 각 장르당 검색할 트랙의 수
completed_workload = 0

for year in year_list:
    for genre in genre_list:
        print(f"{year}_{genre}: 다운로드")    
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []  
        release_date = []
        album_image = []

        artist_popularity = []
        artist_followers =[]

        track_features_list = []
        
        for i in range(0, 10, 10):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=10, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_id.append(t['artists'][0]['id'])
                artist_name.append(t['artists'][0]['name'])
                track_id.append(t['id'])
                track_name.append(t['name'])
                release_date.append(t['album']['release_date'])
                track_popularity.append(t['popularity'])
                try:
                    album_image.append(t['album']['images'][1]['url'])
                except IndexError:
                    album_image.append(None)                
                print(f"음악트랙 데이터 수집 횟수 : {i+1} // 10초 대기")
                time.sleep(3)

                # 아티스트 정보를 요청하는 부분
                artist_info = None
                if t['artists'][0]['id'] in artist_cache:
                    artist_info = artist_cache[t['artists'][0]['id']]
                else:
                    artist_info = sp.artist(t['artists'][0]['id'])
                    artist_cache[t['artists'][0]['id']] = artist_info

                artist_popularity.append(artist_info['popularity'])  # 수정
                artist_followers.append(artist_info['followers']['total'])  # 수정
                print(f"아티스트 데이터 수집 횟수 : {i+1} // 10초 대기")
                time.sleep(3)

                track_features = sp.audio_features([t['id']])
                if track_features:
                    track_features_list.append(track_features[0])
                else:
                    track_features_list.append(None)
                print(f"음악특성 데이터 수집 횟수: {i+1} // 10초 대기")
                time.sleep(3)
                
                completed_workload += 1  # 한 트랙의 처리가 완료될 때마다 증가
                progress = (completed_workload / total_workload) * 100  # 진행 상황을 퍼센트로 계산
                print(f"전체 진행 상황: {progress:.2f}%")
                
        track_df = pd.DataFrame({ 'artist_id': artist_id, 'artist_name': artist_name, 'track_id': track_id, 'track_name': track_name, 'release_date': release_date, 'track_popularity': track_popularity, 'album_image': album_image})

        track_df2 = pd.DataFrame({'artist_id': artist_id, 'artist_name': artist_name, 'artist_popularity': artist_popularity, 'artist_followers': artist_followers})

        track_df3 = pd.DataFrame(track_features_list)
        
        temporary_df = pd.concat([temporary_df, track_df])
        temporary_df2 = pd.concat([temporary_df2, track_df2])  
        temporary_df3 = pd.concat([temporary_df3, track_df3])

temporary_df.drop_duplicates(subset='track_id', inplace=True)

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/test_track.csv', index=False)

temporary_df2.drop_duplicates(subset='artist_id', inplace=True)
#temporary_df2.drop_duplicates(subset='track_id', inplace=True)
temporary_df2 = temporary_df2.drop(columns=['track_id'])

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
temporary_df2.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/test_artist.csv', index=False)

temporary_df3.drop_duplicates(subset='id', inplace=True)
temporary_df3 = temporary_df3.rename(columns={'id': 'track_id'})
temporary_df3 = temporary_df3.drop(columns=['type', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
temporary_df3.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/test_audio.csv', index=False)

end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")


2023_k-pop: 다운로드
음악트랙 데이터 수집 횟수 : 1 // 10초 대기
아티스트 데이터 수집 횟수 : 1 // 10초 대기
음악특성 데이터 수집 횟수: 1 // 10초 대기
전체 진행 상황: 1.00%
음악트랙 데이터 수집 횟수 : 2 // 10초 대기
아티스트 데이터 수집 횟수 : 2 // 10초 대기
음악특성 데이터 수집 횟수: 2 // 10초 대기
전체 진행 상황: 2.00%
음악트랙 데이터 수집 횟수 : 3 // 10초 대기
아티스트 데이터 수집 횟수 : 3 // 10초 대기
음악특성 데이터 수집 횟수: 3 // 10초 대기
전체 진행 상황: 3.00%
음악트랙 데이터 수집 횟수 : 4 // 10초 대기
아티스트 데이터 수집 횟수 : 4 // 10초 대기
음악특성 데이터 수집 횟수: 4 // 10초 대기
전체 진행 상황: 4.00%
음악트랙 데이터 수집 횟수 : 5 // 10초 대기
아티스트 데이터 수집 횟수 : 5 // 10초 대기
음악특성 데이터 수집 횟수: 5 // 10초 대기
전체 진행 상황: 5.00%
음악트랙 데이터 수집 횟수 : 6 // 10초 대기
아티스트 데이터 수집 횟수 : 6 // 10초 대기
음악특성 데이터 수집 횟수: 6 // 10초 대기
전체 진행 상황: 6.00%
음악트랙 데이터 수집 횟수 : 7 // 10초 대기
아티스트 데이터 수집 횟수 : 7 // 10초 대기
음악특성 데이터 수집 횟수: 7 // 10초 대기
전체 진행 상황: 7.00%
음악트랙 데이터 수집 횟수 : 8 // 10초 대기
아티스트 데이터 수집 횟수 : 8 // 10초 대기
음악특성 데이터 수집 횟수: 8 // 10초 대기
전체 진행 상황: 8.00%
음악트랙 데이터 수집 횟수 : 9 // 10초 대기
아티스트 데이터 수집 횟수 : 9 // 10초 대기
음악특성 데이터 수집 횟수: 9 // 10초 대기
전체 진행 상황: 9.00%
음악트랙 데이터 수집 횟수 : 10 // 10초 대기
아티스트 데이터 수집 횟수 : 10 // 10초 대기
음악특성 

KeyError: Index(['track_id'], dtype='object')